In [62]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import heapq

def dijkstra(adj_matrix, start_city, end_city):
    num_cities = len(adj_matrix)
    distances = [sys.maxsize] * num_cities
    visited = [False] * num_cities
    previous = [None] * num_cities

    distances[start_city] = 0

    for _ in range(num_cities):
        min_dist = sys.maxsize
        current_city = None

        for city in range(num_cities):
            if not visited[city] and distances[city] < min_dist:
                min_dist = distances[city]
                current_city = city

        visited[current_city] = True

        if current_city == end_city:
            break

        for neighbor in range(num_cities):
            if (
                not visited[neighbor]
                and adj_matrix[current_city][neighbor] != 0
                and distances[current_city] + adj_matrix[current_city][neighbor] < distances[neighbor]
            ):
                distances[neighbor] = distances[current_city] + adj_matrix[current_city][neighbor]
                previous[neighbor] = current_city

    if distances[end_city] == sys.maxsize:
        return "No path found."

    path = []
    current_city = end_city
    while current_city is not None:
        path.append(current_city)
        current_city = previous[current_city]

    path.reverse()
    return path, distances[end_city]

In [63]:
num_villes = 1000

def generate_weighted_adjacency_matrix(n):
    # Générer une matrice d'adjacence vide
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = random.randint(1, 1000)  # Poids du lien entre deux villes
                adjacency_matrix[i][j] = weight 
                adjacency_matrix[j][i] = weight
                

    return adjacency_matrix

adj_matrix = generate_weighted_adjacency_matrix(num_villes)
print("finished")

finished


In [64]:
NumTruck = 3

population = []
truckJourneys = []
visitedCities = []

initCity = np.random.randint(1,num_villes)

numValidSol = 12

for n in range(numValidSol):
    for i in range(NumTruck):
        truckJourneys.append([initCity])
    visitedCities.append(initCity)
    print("Initial cities : ", truckJourneys)

    while len(visitedCities) < num_villes:
        for j in range(NumTruck):
            curTruck = j
            curCity = truckJourneys[curTruck][-1]
            neighborsCurCity = adj_matrix[curCity]
            CityFound = False
            # Find the indexes of non-null values in neighborsCurCity
            non_zero_indexes = np.nonzero(neighborsCurCity)[0]
            # Filter out the indexes that are in visitedCities
            non_zero_indices = np.setdiff1d(non_zero_indexes, visitedCities)
            if np.isin(non_zero_indices, visitedCities).all():
                non_zero_count = np.count_nonzero(neighborsCurCity)
                probabilities = np.where(np.array(neighborsCurCity) == 0, 0, 1 / non_zero_count)
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
            else:
                size_non_zero_indices = len(non_zero_indices)
                probabilities = np.zeros(len(neighborsCurCity))
                probabilities[non_zero_indices] = 1 / size_non_zero_indices
                probabilities[visitedCities] = 0            
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
                visitedCities.append(selectedCity)

    for j in range(len(truckJourneys)):
        result = dijkstra(adj_matrix, truckJourneys[j][-1], truckJourneys[j][0])
        path, distance = result
        truckJourneys[j] = truckJourneys[j] + path[1:]

    #for j in range(len(truckJourneys)):
        #print("Truck Journey ", j, " : ", truckJourneys[j])
    
    population.append(truckJourneys)
    
    truckJourneys = []
    visitedCities = []

print(population)

Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
Initial cities :  [[724], [724], [724]]
[[[724, 92, 607, 662, 210, 367, 269, 544, 41, 553, 285, 736, 622, 483, 672, 44, 352, 852, 282, 653, 238, 936, 334, 16, 735, 805, 649, 918, 281, 714, 47, 292, 819, 945, 205, 984, 283, 47, 197, 429, 751, 334, 149, 36, 796, 573, 781, 463, 28, 691, 108, 314, 134, 207, 865, 287, 513, 896, 848, 123, 843, 492, 444, 309, 156, 58, 39, 10, 347, 540, 428, 14, 362, 131, 255, 480, 208, 704, 52, 308, 59, 620, 310, 922, 866, 333, 601, 808, 609, 504, 557, 962, 129, 633, 162, 319, 456, 317, 276, 390, 474, 113, 65, 192, 538, 619, 45

In [65]:
# Function to calculate the fitness of an individual
def calculate_fitness(individual):
    distances = [0] * len(individual)
    for i, path in enumerate(individual):
        path_distance = 0
        for j in range(len(path) - 1):
            start_city = path[j]
            end_city = path[j + 1]
            path_distance += adj_matrix[start_city][end_city]
        distances[i] = path_distance
    difference = max(distances) - min(distances)
    return difference

for individual in population:
    fitness = calculate_fitness(individual)
    print(fitness)
    


6473
13944
7838
13420
16999
8218
8041
13225
9749
10512
1982
25590


## Algorithme génétique

In [90]:
# Constants
POPULATION_SIZE = numValidSol
NUM_GENERATIONS = 100
MUTATION_RATE = 0.5

In [91]:
def repair_non_neighboring_cities(path, index):
    if index+1 <= len(path)-1:
        city1 = path[index]
        city2 = path[index+1]
        if city1 == city2:
            del path[index]
        else:
            adj_value = adj_matrix[city1][city2]
            if adj_value == 0:
                result = dijkstra(adj_matrix, city1, city2)
                path_to_add, distance = result
                path[index+1:index+1] = path_to_add[1:-1]
    return path

In [92]:
def repair_missing_cities(individual):
    visited_cities = set(city for path in individual for city in path)
    missing_cities = set(range(len(adj_matrix))) - visited_cities
    while len(missing_cities) > 0:
        missing_city = missing_cities.pop()
        for path in individual:
            for i in range(len(path)-1):
                if adj_matrix[missing_city][i] != 0 and adj_matrix[missing_city][i+1] != 0:
                    path.insert(i+1, missing_city)
                    missing_city = -1
                    break
            if missing_city == -1:
                break
        if missing_city != -1:
            neighborCities = [i for i, val in enumerate(adj_matrix[missing_city]) if val != 0 and i not in missing_cities]
            neighborCity = np.random.choice(neighborCities)
            for path in individual:
                try:
                    indexneighborCity = path.index(neighborCity)
                    path.insert(indexneighborCity+1, missing_city)
                    result = dijkstra(adj_matrix, indexneighborCity+1, indexneighborCity+2)
                    path_to_add, distance = result
                    index_to_add = indexneighborCity+2
                    path[index_to_add:index_to_add] = path_to_add[1:-1]
                except:
                    pass
    return individual

In [93]:
# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = [[] for _ in range(len(parent1))]
    for i in range(len(parent1)):
        cut_point = random.randint(0, len(parent1[i]) - 1)
        child[i] = parent1[i][:cut_point] + parent2[i][cut_point:]
        child[i] = repair_non_neighboring_cities(child[i], cut_point)

    child = repair_missing_cities(child)  # Repair the child to ensure it visits all cities
    return child

In [94]:
# Function to perform mutation on an individual
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            swap_indices = random.sample(range(1, len(individual[i]) - 1), 2)
            individual[i][swap_indices[0]], individual[i][swap_indices[1]] = \
                individual[i][swap_indices[1]], individual[i][swap_indices[0]]
            for index in swap_indices:
                repair_non_neighboring_cities(individual[i], index)
                if index > 0:
                    repair_non_neighboring_cities(individual[i], index-1)
    return individual

In [95]:
# Main genetic algorithm loop
for generation in range(NUM_GENERATIONS):
    # Calculate fitness for each individual
    fitness_scores = [calculate_fitness(individual) for individual in population]
    
    # Find the best individual in the population
    best_fitness = min(fitness_scores)
    best_individual = population[fitness_scores.index(best_fitness)]
    
    print(f"Generation {generation + 1}: Best Distance = {best_fitness}")
    
    # Select parents for crossover using tournament selection
    parents = []
    for _ in range(POPULATION_SIZE // 2):
        tournament = random.sample(range(POPULATION_SIZE), 5)
        tournament_fitness = [fitness_scores[i] for i in tournament]
        winner = tournament[tournament_fitness.index(min(tournament_fitness))]
        parents.append(population[winner])
    
    # Create new population through crossover and mutation
    new_population = []
    for i in range(0, len(parents), 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]
        child1 = crossover(parent1, parent2)
        child2 = crossover(parent2, parent1)
        new_population.extend([mutate(child1), mutate(child2)])
        
    population.extend(new_population)
    fitness_scores.extend([calculate_fitness(individual) for individual in new_population])

    
    # Combine the elements of lists population and fitness_scores
    combined = zip(population, fitness_scores)

    # Sort the pairs based on the values in list fitness_scores
    sorted_pairs = sorted(combined, key=lambda x: x[1])

    # Extract the desired values
    population = [pair[0] for pair in sorted_pairs[:POPULATION_SIZE]]
    fitness_scores = [pair[1] for pair in sorted_pairs[:POPULATION_SIZE]]
    
    
# Print the best solution
best_fitness = min(fitness_scores)
best_individual = population[fitness_scores.index(best_fitness)]
print(f"\nBest Solution: {best_individual}")
print(f"Best Distance: {best_fitness}")

Generation 1: Best Distance = 1982
Generation 2: Best Distance = 1982
Generation 3: Best Distance = 1982
Generation 4: Best Distance = 1982
Generation 5: Best Distance = 1982
Generation 6: Best Distance = 1982
Generation 7: Best Distance = 1982
Generation 8: Best Distance = 1982
Generation 9: Best Distance = 1982
Generation 10: Best Distance = 1982
Generation 11: Best Distance = 1982
Generation 12: Best Distance = 1982
Generation 13: Best Distance = 1982
Generation 14: Best Distance = 1982
Generation 15: Best Distance = 1982
Generation 16: Best Distance = 1982
Generation 17: Best Distance = 1982
Generation 18: Best Distance = 1982
Generation 19: Best Distance = 1982
Generation 20: Best Distance = 1982
Generation 21: Best Distance = 1982
Generation 22: Best Distance = 1982
Generation 23: Best Distance = 1982
Generation 24: Best Distance = 1982
Generation 25: Best Distance = 1982
Generation 26: Best Distance = 1982
Generation 27: Best Distance = 1982
Generation 28: Best Distance = 1982
G